In [1]:
import requests
from bs4 import BeautifulSoup

## Q1.1

In [2]:
# Find the latest press releases pages
url = "https://press.un.org/en"
response = requests.get(url)

results_page = BeautifulSoup(response.content, 'lxml')
press_release = results_page.find_all('div', {'class': 'view-footer'})[1]

press_release_first_page = press_release.find('a').get('href')
press_release_response = requests.get(press_release_first_page)
press_release_results_first_page = BeautifulSoup(press_release_response.content, 'lxml')

In [3]:
press_release_with_crisis = []

def extract_press_release_with_crisis(results_page):
    seed_url = "https://press.un.org"
    all_urls = results_page.find_all('h3')

    for i in range(0, len(all_urls)):
        press_release_url = seed_url + all_urls[i].find('a').get('href')

        press_release = requests.get(press_release_url)
        press_release_page = BeautifulSoup(press_release.content, 'lxml')

        paragraphs = press_release_page.find_all('p')
        for paragraph in paragraphs:
            if 'crisis' in paragraph.text.lower() and press_release_url not in press_release_with_crisis:
                press_release_with_crisis.append(press_release_url)
            else:
                next
    return press_release_with_crisis
    

In [4]:
current_extracted = extract_press_release_with_crisis(press_release_results_first_page)
current_extracted

['https://press.un.org/en/2023/sgsm21967.doc.htm']

In [5]:
if len(current_extracted) < 10:
    press_release_response = requests.get(press_release_first_page)
    press_release_results_page = BeautifulSoup(press_release_response.content, 'lxml')

    pages = press_release_results_page.find_all('li', {'class': 'page-item pager__item'})
    
    for i in range(0, len(pages)):
        next_page = press_release_first_page + pages[i].find('a').get('href')

        new_page_response = requests.get(next_page)
        press_release_results_new_page = BeautifulSoup(new_page_response.content, 'lxml')

        current_extracted = extract_press_release_with_crisis(press_release_results_new_page)  

        if len(current_extracted) > 10:
            break      

In [6]:
current_extracted

['https://press.un.org/en/2023/sgsm21967.doc.htm',
 'https://press.un.org/en/2023/sga2230.doc.htm',
 'https://press.un.org/en/2023/dsgsm1877.doc.htm',
 'https://press.un.org/en/2023/sgsm21959.doc.htm',
 'https://press.un.org/en/2023/sgsm21956.doc.htm',
 'https://press.un.org/en/2023/sgsm21952.doc.htm',
 'https://press.un.org/en/2023/sgsm21951.doc.htm',
 'https://press.un.org/en/2023/sgsm21950.doc.htm',
 'https://press.un.org/en/2023/sgsm21947.doc.htm',
 'https://press.un.org/en/2023/sgsm21945.doc.htm',
 'https://press.un.org/en/2023/dsgsm1874.doc.htm',
 'https://press.un.org/en/2023/dsgsm1873.doc.htm']

## Save Press Releases to txt Files

In [12]:
for i in range(0, len(current_extracted)):
    URL = current_extracted[i]

    response = requests.get(URL)
    soup = BeautifulSoup(response.content, 'lxml')

    title = soup.find('h1').get_text()

    paragraphs = [title]

    text = soup.find_all('p')

    for j in range(0, len(text) - 2):
        paragraphs.append(text[j].get_text())

        article = '\n'.join(paragraphs)

        file_name = f'1_{i + 1}.txt'
        with open(file_name, 'w', encoding = 'utf-8') as file:
            file.write(article)